In [ ]:
import pandas as pd
from model import LinearModel, MedianHeuristic, PvalueLog, ConfIntHSIC
from hsic import hsic_perm_test
import patsy
import numpy as np
from functools import partial
from statsmodels.sandbox.regression.gmm import LinearIVGMM
from statsmodels.formula.api import ols
import itertools
from joblib import Parallel, delayed
from kernel import RBFKernel, CategoryKernel, ProductKernel3, ProductKernel2
from torch.utils.data import TensorDataset
from linearmodels.iv.model import IV2SLS
import torch
from utils import *
import warnings
warnings.simplefilter("ignore")

np.random.seed(0)

In [ ]:
df = pd.read_csv("card.csv", index_col=0)

In [ ]:
OLS = ols("lwage ~ -1 + educ + C(black) + C(smsa66) + C(exp_bin) + C(south66)", df).fit()

In [ ]:
print(OLS.params['educ'])

In [ ]:
OLS.conf_int()

In [ ]:
def invert_test(p, X, W, Y, Z, kernel_z, method='gamma'):
    Y_hat = Y - p * X
    se_callback = MedianHeuristic()
    pval_callback = PvalueLog()
    
    batch_size = 256
    
    kernel_e = RBFKernel(sigma=1)
    hsic_net = LinearModel(input_dim=W.shape[1],
                           lr=1e-2,
                           lmd=0.0,
                           kernel_e=kernel_e,
                           kernel_z=kernel_z,
                           bias=False)
    
    trainloader = torch.utils.data.DataLoader(TensorDataset(to_torch(W), to_torch(Y_hat), to_torch(Z)),
                                              batch_size=batch_size,
                                              shuffle=True, num_workers=0)
    
    max_epoch = 50
    hsic_net = fit_restart(trainloader, hsic_net, pval_callback, max_epoch, 
                           se_callback, num_restart=1, verbose=False)
    
    res = Y_hat - hsic_net(to_torch(W)).detach().numpy()
    
    sigma_e = med_sigma(res)
    kernel_e = RBFKernel(sigma=sigma_e)
    
    if method == 'gamma':
        if kernel_z.__class__ == CategoryKernel:
            kernels = ['gaussian', 'discrete']
        else:
            kernels = ['gaussian', 'gaussian']
        
        pval, hsic = dhsic_test(res, Z, kernels, 
                         statistics=True, method='gamma')
    elif method == 'permu':
        pval, hsic = hsic_perm_test(res, Z, kernel_e, kernel_z, B=100)
        
    return p, pval, hsic

In [ ]:
W = np.asarray(patsy.dmatrix("~ -1 + C(black) + C(smsa66) + C(south66) + C(exp_bin)", data=df, return_type='matrix'))
Z = np.asarray(patsy.dmatrix("~ -1 + C(nearc4)", data=df, return_type='matrix'))
Z = np.hstack([Z, W])
X = df['educ']
Y = df['lwage']

In [ ]:
kernel_z = ProductKernel2(CategoryKernel(one_hot=False),
                          RBFKernel(1),
                          [0, 1],
                          [2, 3, 4, 5, 6, 7, 8])

In [ ]:
param_range = np.linspace(0.03, 0.23, 64)

pval_ret = Parallel(n_jobs=-1)(delayed(invert_test)(p, X, W, Y, Z, kernel_z, method='permu') for p in param_range)

In [ ]:
pval_df = pd.DataFrame(pval_ret, columns=['param', 'pval', 'stat'])
pval_df.to_csv("card_pval_df.csv", index=False)

In [ ]:
point_estimate = pval_df.param[pval_df.stat == pval_df.stat.min()].iloc[0]

In [ ]:
accept_df = pval_df.query('pval >= 0.05')
conf_int = accept_df.param.min(), accept_df.param.max()

In [ ]:
point_estimate

In [ ]:
conf_int